## Import Required Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import anndata as ad
import scanpy as sc
from PIL import Image
import tifffile as tifi
from IPython.display import IFrame
from pathlib import Path
import os; from tqdm import tqdm

In [2]:
import os

# Define base directory
base_dir = '/scratch/Projects/IMMUNEX/enact_cache'

# Find all subfolders starting with "run6"
run_folders = [
    os.path.join(base_dir, folder)
    for folder in os.listdir(base_dir)
    if folder.startswith("run6") and os.path.isdir(os.path.join(base_dir, folder))
]

run_folders

['/scratch/Projects/IMMUNEX/enact_cache/run6_IMMUNEX007',
 '/scratch/Projects/IMMUNEX/enact_cache/run6_IMMUNEX011',
 '/scratch/Projects/IMMUNEX/enact_cache/run6_IMMUNEX001',
 '/scratch/Projects/IMMUNEX/enact_cache/run6_IMMUNEX016',
 '/scratch/Projects/IMMUNEX/enact_cache/run6_IMMUNEX006',
 '/scratch/Projects/IMMUNEX/enact_cache/run6_IMMUNEX012',
 '/scratch/Projects/IMMUNEX/enact_cache/run6_IMMUNEX005',
 '/scratch/Projects/IMMUNEX/enact_cache/run6_IMMUNEX009',
 '/scratch/Projects/IMMUNEX/enact_cache/run6_IMMUNEX010',
 '/scratch/Projects/IMMUNEX/enact_cache/run6_IMMUNEX017',
 '/scratch/Projects/IMMUNEX/enact_cache/run6_IMMUNEX004',
 '/scratch/Projects/IMMUNEX/enact_cache/run6_IMMUNEX014',
 '/scratch/Projects/IMMUNEX/enact_cache/run6_IMMUNEX013',
 '/scratch/Projects/IMMUNEX/enact_cache/run6_IMMUNEX008',
 '/scratch/Projects/IMMUNEX/enact_cache/run6_IMMUNEX018',
 '/scratch/Projects/IMMUNEX/enact_cache/run6_IMMUNEX015',
 '/scratch/Projects/IMMUNEX/enact_cache/run6_IMMUNEX003',
 '/scratch/Pro

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

all_adatas = []


for sample_path in tqdm(run_folders):
    try:
        # path (basedir)
        output_path = Path(sample_path)
        
        # Anndata object path
        cells_adata_path = output_path / "tmap/weighted_by_cluster|celltypist_cells_adata.h5"
        
        # Cropped whole slide image
        wsi_path = output_path / "tmap/wsi.tif"
        
        # Cell boundaries image
        cells_layer_path = output_path / "tmap/cells_layer.png"
    
        # Loading Anndata object
        adata = sc.read_h5ad(cells_adata_path)
        adata.obs['sample_path'] = sample_path
        adata.obs['sample'] = os.path.basename(sample_path)


    except:
        pass
    
    all_adatas.append(adata)



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:08<00:00,  2.06it/s]


In [6]:
import anndata as ad

merged_adata = ad.concat(all_adatas , join="outer")

print(merged_adata.shape)
display(merged_adata.obs.sample(5))

display(merged_adata.obs['sample'].unique())

(7829536, 5759)


/scratch/IMMUNEX/pipelines/ENACT/enact-pipeline/enact_env/home/mounim/miniconda3/bin/python3/lib/python3.10/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


,cell_type,patch_id,sample_path,sample
id,,,,
ID_240427,epithelial cells,patch_1_1.csv,/scratch/Projects/IMMUNEX/enact_cache/run6_IMM...,run6_IMMUNEX008
ID_97524,double-positive thymocytes,patch_0_0.csv,/scratch/Projects/IMMUNEX/enact_cache/run6_IMM...,run6_IMMUNEX017
ID_122000,double-positive thymocytes,patch_0_0.csv,/scratch/Projects/IMMUNEX/enact_cache/run6_IMM...,run6_IMMUNEX012
ID_289172,double-positive thymocytes,patch_2_1.csv,/scratch/Projects/IMMUNEX/enact_cache/run6_IMM...,run6_IMMUNEX018
ID_255160,double-positive thymocytes,patch_0_0.csv,/scratch/Projects/IMMUNEX/enact_cache/run6_IMM...,run6_IMMUNEX003


array(['run6_IMMUNEX007', 'run6_IMMUNEX011', 'run6_IMMUNEX001',
       'run6_IMMUNEX016', 'run6_IMMUNEX006', 'run6_IMMUNEX012',
       'run6_IMMUNEX005', 'run6_IMMUNEX009', 'run6_IMMUNEX017',
       'run6_IMMUNEX004', 'run6_IMMUNEX014', 'run6_IMMUNEX013',
       'run6_IMMUNEX008', 'run6_IMMUNEX018', 'run6_IMMUNEX015',
       'run6_IMMUNEX003'], dtype=object)

In [16]:
import os
import pandas as pd
from collections import defaultdict

base_dir = "/scratch/Projects/IMMUNEX/results/enact_results"

# Expected files and folders
expected_top_level = [
    "cells_df.csv", "cells_layer.png", "chunks", "ENACT.log",
    "nuclei_df.csv", "stardist_labels.npz"
]
expected_chunks_subdirs = ["bins_gdf", "cells_gdf", "weighted_by_cluster"]
expected_weighted_by_cluster_subdirs = ["bin_to_cell_assign", "cell_ix_lookup", "celltypist_results"]
expected_bin_to_cell_assign_files = ["patch_0_0.csv", "spatial_join_patch_0_0.csv"]

# Reverse mapping: missing item → list of folders missing it
missing_items_report = defaultdict(list)

for subfolder in os.listdir(base_dir):
    if 'NoExpNaive' in subfolder:
        subfolder_path = os.path.join(base_dir, subfolder)
        if not os.path.isdir(subfolder_path):
            continue

        # Check top-level
        for name in expected_top_level:
            if not os.path.exists(os.path.join(subfolder_path, name)):
                missing_items_report[name].append(subfolder)

        # chunks subdirs
        chunks_path = os.path.join(subfolder_path, "chunks")
        for name in expected_chunks_subdirs:
            if not os.path.exists(os.path.join(chunks_path, name)):
                missing_items_report[f"chunks/{name}"].append(subfolder)

        # weighted_by_cluster subdirs
        cluster_path = os.path.join(chunks_path, "weighted_by_cluster")
        for name in expected_weighted_by_cluster_subdirs:
            if not os.path.exists(os.path.join(cluster_path, name)):
                missing_items_report[f"chunks/weighted_by_cluster/{name}"].append(subfolder)

        # bin_to_cell_assign files
        bin_assign_path = os.path.join(cluster_path, "bin_to_cell_assign")
        for fname in expected_bin_to_cell_assign_files:
            if not os.path.exists(os.path.join(bin_assign_path, fname)):
                missing_items_report[f"chunks/weighted_by_cluster/bin_to_cell_assign/{fname}"].append(subfolder)

# Convert to DataFrame (each row = missing item, each cell = list of runs)
output = pd.DataFrame.from_dict(missing_items_report, orient="index")
output

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
cells_df.csv,250627_NoExpNaive-IMMUNEX011,250627_NoExpNaive-IMMUNEX015,250627_NoExpNaive-IMMUNEX007,250627_NoExpNaive-IMMUNEX009,250627_NoExpNaive-IMMUNEX004,250627_NoExpNaive-IMMUNEX010,250627_NoExpNaive-IMMUNEX018,250627_NoExpNaive-IMMUNEX012,250627_NoExpNaive-IMMUNEX002,250627_NoExpNaive-IMMUNEX006,250627_NoExpNaive-IMMUNEX003,250627_NoExpNaive-IMMUNEX008,250627_NoExpNaive-IMMUNEX014,250627_NoExpNaive-IMMUNEX013,250627_NoExpNaive-IMMUNEX005,250627_NoExpNaive-IMMUNEX016,250627_NoExpNaive-IMMUNEX001,250627_NoExpNaive-IMMUNEX017
ENACT.log,250627_NoExpNaive-IMMUNEX011,250627_NoExpNaive-IMMUNEX015,250627_NoExpNaive-IMMUNEX009,250627_NoExpNaive-IMMUNEX004,250627_NoExpNaive-IMMUNEX010,250627_NoExpNaive-IMMUNEX018,250627_NoExpNaive-IMMUNEX012,250627_NoExpNaive-IMMUNEX002,250627_NoExpNaive-IMMUNEX014,250627_NoExpNaive-IMMUNEX013,250627_NoExpNaive-IMMUNEX005,250627_NoExpNaive-IMMUNEX016,250627_NoExpNaive-IMMUNEX001,250627_NoExpNaive-IMMUNEX017,None,None,None,None
chunks/weighted_by_cluster,250627_NoExpNaive-IMMUNEX011,250627_NoExpNaive-IMMUNEX015,250627_NoExpNaive-IMMUNEX007,250627_NoExpNaive-IMMUNEX009,250627_NoExpNaive-IMMUNEX004,250627_NoExpNaive-IMMUNEX010,250627_NoExpNaive-IMMUNEX018,250627_NoExpNaive-IMMUNEX012,250627_NoExpNaive-IMMUNEX002,250627_NoExpNaive-IMMUNEX006,250627_NoExpNaive-IMMUNEX003,250627_NoExpNaive-IMMUNEX008,250627_NoExpNaive-IMMUNEX014,250627_NoExpNaive-IMMUNEX013,250627_NoExpNaive-IMMUNEX005,250627_NoExpNaive-IMMUNEX016,250627_NoExpNaive-IMMUNEX001,250627_NoExpNaive-IMMUNEX017
chunks/weighted_by_cluster/bin_to_cell_assign,250627_NoExpNaive-IMMUNEX011,250627_NoExpNaive-IMMUNEX015,250627_NoExpNaive-IMMUNEX007,250627_NoExpNaive-IMMUNEX009,250627_NoExpNaive-IMMUNEX004,250627_NoExpNaive-IMMUNEX010,250627_NoExpNaive-IMMUNEX018,250627_NoExpNaive-IMMUNEX012,250627_NoExpNaive-IMMUNEX002,250627_NoExpNaive-IMMUNEX006,250627_NoExpNaive-IMMUNEX003,250627_NoExpNaive-IMMUNEX008,250627_NoExpNaive-IMMUNEX014,250627_NoExpNaive-IMMUNEX013,250627_NoExpNaive-IMMUNEX005,250627_NoExpNaive-IMMUNEX016,250627_NoExpNaive-IMMUNEX001,250627_NoExpNaive-IMMUNEX017
chunks/weighted_by_cluster/cell_ix_lookup,250627_NoExpNaive-IMMUNEX011,250627_NoExpNaive-IMMUNEX015,250627_NoExpNaive-IMMUNEX007,250627_NoExpNaive-IMMUNEX009,250627_NoExpNaive-IMMUNEX004,250627_NoExpNaive-IMMUNEX010,250627_NoExpNaive-IMMUNEX018,250627_NoExpNaive-IMMUNEX012,250627_NoExpNaive-IMMUNEX002,250627_NoExpNaive-IMMUNEX006,250627_NoExpNaive-IMMUNEX003,250627_NoExpNaive-IMMUNEX008,250627_NoExpNaive-IMMUNEX014,250627_NoExpNaive-IMMUNEX013,250627_NoExpNaive-IMMUNEX005,250627_NoExpNaive-IMMUNEX016,250627_NoExpNaive-IMMUNEX001,250627_NoExpNaive-IMMUNEX017
chunks/weighted_by_cluster/celltypist_results,250627_NoExpNaive-IMMUNEX011,250627_NoExpNaive-IMMUNEX015,250627_NoExpNaive-IMMUNEX007,250627_NoExpNaive-IMMUNEX009,250627_NoExpNaive-IMMUNEX004,250627_NoExpNaive-IMMUNEX010,250627_NoExpNaive-IMMUNEX018,250627_NoExpNaive-IMMUNEX012,250627_NoExpNaive-IMMUNEX002,250627_NoExpNaive-IMMUNEX006,250627_NoExpNaive-IMMUNEX003,250627_NoExpNaive-IMMUNEX008,250627_NoExpNaive-IMMUNEX014,250627_NoExpNaive-IMMUNEX013,250627_NoExpNaive-IMMUNEX005,250627_NoExpNaive-IMMUNEX016,250627_NoExpNaive-IMMUNEX001,250627_NoExpNaive-IMMUNEX017
chunks/weighted_by_cluster/bin_to_cell_assign/patch_0_0.csv,250627_NoExpNaive-IMMUNEX011,250627_NoExpNaive-IMMUNEX015,250627_NoExpNaive-IMMUNEX007,250627_NoExpNaive-IMMUNEX009,250627_NoExpNaive-IMMUNEX004,250627_NoExpNaive-IMMUNEX010,250627_NoExpNaive-IMMUNEX018,250627_NoExpNaive-IMMUNEX012,250627_NoExpNaive-IMMUNEX002,250627_NoExpNaive-IMMUNEX006,250627_NoExpNaive-IMMUNEX003,250627_NoExpNaive-IMMUNEX008,250627_NoExpNaive-IMMUNEX014,250627_NoExpNaive-IMMUNEX013,250627_NoExpNaive-IMMUNEX005,250627_NoExpNaive-IMMUNEX016,250627_NoExpNaive-IMMUNEX001,250627_NoExpNaive-IMMUNEX017
chunks/weighted_by_cluster/bin_to_cell_assign/spatial_join_patch_0_0.csv,250627_NoExpNaive-IMMUNEX011,250627_NoExpNaive-IMMUNEX015,250627_NoExpNaive-IMMU